In [28]:
import pandas as pd
import numpy as np
from tensorflow import keras
from keras import layers
import matplotlib.pyplot as plt 

In [29]:
# PATH = "../kaggle/data/"
PATH = "/kaggle/input/isic-2024-challenge/"


In [30]:
df_0 = pd.read_csv(PATH + "train-metadata.csv")

/tmp/ipykernel_33/1877868789.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df_0 = pd.read_csv(PATH + "train-metadata.csv")


In [31]:
# Clean and preprocess data for training and testing 

cols_to_drop = ["isic_id", "patient_id", "mel_thick_mm", "iddx_full", "iddx_1", "iddx_2", "iddx_3", "iddx_4", "iddx_5", "attribution", "copyright_license", "tbp_lv_location", "tbp_lv_location_simple", "tbp_lv_dnn_lesion_confidence", "anatom_site_general", "image_type", "tbp_tile_type"]
df_0.drop(columns=cols_to_drop, inplace=True)
df_0["sex"] = df_0["sex"].replace({"male": 1, "female": 0})

# replace all NaNs with np.nan
df_0.replace("", np.nan, inplace=True)

# Drop any row with a NaN value
df_0.dropna(inplace=True)

# Drop any column with a string vallue 
for col in df_0.columns:
    if df_0[col].dtype == "object":
        df_0.drop(columns=[col], inplace=True)

# clean up for training, split into train and test
train_frac = 0.8 
train_size = int(train_frac * len(df_0))

train = df_0[:train_size]
x_train = df_0.drop(columns=["target"])
y_train = df_0["target"]

test = df_0[train_size:]
x_test = df_0.drop(columns=["target"])
y_test = df_0["target"]

/tmp/ipykernel_33/1864818803.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_0["sex"] = df_0["sex"].replace({"male": 1, "female": 0})


In [32]:
# Define the NN 
inputs = keras.Input(shape=(len(x_train.keys()),)) # One input for each feature in the dataset 

# Add two hidden layers with 64 units each and ReLU activation
x = layers.Dense(64, activation="relu")(inputs)
x = layers.Dense(64, activation="relu")(x)

# Define the output layer with a single unit and sigmoid activation
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="isic-v0")


In [33]:
model.compile(
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=[keras.metrics.BinaryAccuracy()],
)

history = model.fit(x_train, y_train, batch_size=512, epochs=10, validation_split=0.2)

test_scores = model.evaluate(x_test, y_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/backend/tensorflow/nn.py:681: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 987ms/step - binary_accuracy: 0.0488 - loss: 44.1796 - val_binary_accuracy: 0.9091 - val_loss: 0.1275
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - binary_accuracy: 1.0000 - loss: 0.0015 - val_binary_accuracy: 0.9091 - val_loss: 0.1105
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - binary_accuracy: 1.0000 - loss: 9.5178e-04

/opt/conda/lib/python3.10/site-packages/keras/src/backend/tensorflow/nn.py:681: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - binary_accuracy: 1.0000 - loss: 9.5178e-04 - val_binary_accuracy: 0.9091 - val_loss: 0.0993
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - binary_accuracy: 1.0000 - loss: 7.0029e-04 - val_binary_accuracy: 0.9091 - val_loss: 0.0909
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - binary_accuracy: 1.0000 - loss: 5.5198e-04 - val_binary_accuracy: 0.9091 - val_loss: 0.0840
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - binary_accuracy: 1.0000 - loss: 4.5301e-04 - val_binary_accuracy: 0.9091 - val_loss: 0.0783
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - binary_accuracy: 1.0000 - loss: 3.8192e-04 - val_binary_accuracy: 0.9091 - val_loss: 0.0733
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - binary_accuracy: 1.0000 - loss: 3.2823e-04 - val_binary_accuracy: 0.9091 - val_loss: 0.0689
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - binary_accuracy: 1.0000 - loss: 2.8620e-04 - val_binary_accuracy: 0.9091 - val_loss: 0.0650
Epoch 10/1

In [34]:
test_df = pd.read_csv(PATH + "test-metadata.csv")
isic_ids = test_df["isic_id"]

try: 
    test_df.drop(columns=cols_to_drop, inplace=True)
except KeyError: 
    pass

test_df.replace("", np.nan, inplace=True)
test_df["sex"] = test_df["sex"].replace({"male": 1, "female": 0})
test_df.dropna(inplace=True)

# Drop any column with a string vallue 
for col in test_df.columns:
    if test_df[col].dtype == "object":
        test_df.drop(columns=[col], inplace=True)

/tmp/ipykernel_33/554657857.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_df["sex"] = test_df["sex"].replace({"male": 1, "female": 0})


In [35]:
print(len(test_df.keys()), len(x_train.keys()))

35 35


In [36]:
preds = model.predict(test_df)

df_sub = pd.DataFrame({"isic_id": isic_ids, "target": preds[:,0]})
df_sub.to_csv("submission.csv", index=False)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step


In [37]:
df_sub

,isic_id,target
0,ISIC_0015657,0.999984
1,ISIC_0015729,1.000000
2,ISIC_0015740,1.000000
